## LightGBM-Optunaのサンプルコード（train, valid, test使用）

In [1]:
%load_ext lab_black

In [2]:
# ライブラリーのインポート
import os

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# ボストンの住宅価格データ
from sklearn.datasets import load_boston

# 前処理
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# LightGBM
import lightgbm as lgb

# Optuna
import optuna

# 評価指標
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [3]:
print(lgb.__version__)

3.1.0


In [4]:
# データセットの読込み
boston = load_boston()

# 説明変数の格納
df = pd.DataFrame(boston.data, columns=boston.feature_names)
# 目的変数の追加
df["MEDV"] = boston.target

# データの中身を確認
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


#### 前処理

In [5]:
# ランダムシード値
RANDOM_STATE = 10

# 学習データと評価データの割合
TEST_SIZE = 0.2

# 学習データと評価データを作成
x_train, x_test, y_train, y_test = train_test_split(
    df.iloc[:, 0 : df.shape[1] - 1],
    df.iloc[:, df.shape[1] - 1],
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
)

# trainのデータセットの2割をモデル学習時のバリデーションデータとして利用する
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

In [6]:
def objective(trial):

    params = {
        "objective": "mean_squared_error",
        "metric": "rmse",
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    # LightGBMを利用するのに必要なフォーマットに変換
    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_eval = lgb.Dataset(x_valid, y_valid, reference=lgb_train)

    # 学習の経過を保存する箱
    evaluation_results = {}

    model = lgb.train(
        params,
        lgb_train,
        num_boost_round=200,
        valid_names=["train", "valid"],
        valid_sets=[lgb_train, lgb_eval],
        evals_result=evaluation_results,
    )

    preds = model.predict(x_valid)
    mae = mean_absolute_error(y_valid, preds)

    return mae

In [7]:
%%time
# optunaで最適値を見つける
# create_studyメソッドの引数"sampler"にサンプラーと乱数シードを指定
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study.optimize(objective, n_trials=50)

[I 2021-06-22 14:51:45,991] A new study created in memory with name: no-name-40af2c63-8a3a-4024-b4e8-518f15c5258c


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000096 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 807
[LightGBM] [Info] Number of data points in the train set: 323, number of used features: 12
[LightGBM] [Info] Start training from score 21.968731
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	train's rmse: 8.49905	valid's rmse: 6.64307
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	train's rmse: 7.94829	valid's rmse: 6.23289
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	train's rmse: 7.41287	valid's rmse: 5.80693
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	train's rmse: 6.92764	valid's rmse: 5.45293
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	train's rmse: 6.55394	

[I 2021-06-22 14:51:46,347] Trial 0 finished with value: 2.5374505963248395 and parameters: {'lambda_l1': 0.08747537025773001, 'lambda_l2': 1.537331564587801e-08, 'num_leaves': 163, 'feature_fraction': 0.8492823295231671, 'bagging_fraction': 0.6991042073815543, 'bagging_freq': 2, 'min_child_samples': 24}. Best is trial 0 with value: 2.5374505963248395.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[153]	train's rmse: 1.97963	valid's rmse: 3.62174
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[154]	train's rmse: 1.97826	valid's rmse: 3.61478
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[155]	train's rmse: 1.97104	valid's rmse: 3.62439
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[156]	train's rmse: 1.9651	valid's rmse: 3.62572
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[157]	train's rmse: 1.96016	valid's rmse: 3.63855
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[158]	train's rmse: 1.95163	valid's rmse: 3.62758
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[159]	train's rmse: 1.94386	valid's rmse: 3.61649
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[160]	train's rmse: 1.93559	valid's rmse: 3.60876
[

[I 2021-06-22 14:51:46,647] Trial 1 finished with value: 2.7739694805377177 and parameters: {'lambda_l1': 0.0699481785242808, 'lambda_l2': 3.32657660618516e-07, 'num_leaves': 24, 'feature_fraction': 0.8112158910206784, 'bagging_fraction': 0.972036007716962, 'bagging_freq': 1, 'min_child_samples': 54}. Best is trial 0 with value: 2.5374505963248395.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[151]	train's rmse: 0.5467	valid's rmse: 2.87141
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[152]	train's rmse: 0.544394	valid's rmse: 2.86825
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[153]	train's rmse: 0.541338	valid's rmse: 2.8666
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[154]	train's rmse: 0.532716	valid's rmse: 2.86771
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[155]	train's rmse: 0.529399	valid's rmse: 2.87005
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[156]	train's rmse: 0.520131	valid's rmse: 2.8745
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[157]	train's rmse: 0.517601	valid's rmse: 2.87126
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[158]	train's rmse: 0.512137	valid's rmse: 2.86

[I 2021-06-22 14:52:09,022] Trial 48 finished with value: 2.3370314192615482 and parameters: {'lambda_l1': 0.06176369264321309, 'lambda_l2': 0.15823899741634775, 'num_leaves': 150, 'feature_fraction': 0.5117618318988791, 'bagging_fraction': 0.7033120121958578, 'bagging_freq': 1, 'min_child_samples': 14}. Best is trial 44 with value: 1.984056732857496.


[183]	train's rmse: 1.05787	valid's rmse: 3.18333
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[184]	train's rmse: 1.05327	valid's rmse: 3.18761
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[185]	train's rmse: 1.04897	valid's rmse: 3.18357
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[186]	train's rmse: 1.04294	valid's rmse: 3.18436
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[187]	train's rmse: 1.03944	valid's rmse: 3.18191
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[188]	train's rmse: 1.03748	valid's rmse: 3.17943
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[189]	train's rmse: 1.03074	valid's rmse: 3.18451
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[190]	train's rmse: 1.02747	valid's rmse: 3.18054
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2021-06-22 14:52:09,490] Trial 49 finished with value: 2.377703701166847 and parameters: {'lambda_l1': 2.5218333554863165e-05, 'lambda_l2': 0.06104984364003374, 'num_leaves': 205, 'feature_fraction': 0.6895682808099396, 'bagging_fraction': 0.7379531515265398, 'bagging_freq': 2, 'min_child_samples': 20}. Best is trial 44 with value: 1.984056732857496.


[148]	train's rmse: 1.64154	valid's rmse: 3.26104
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[149]	train's rmse: 1.63209	valid's rmse: 3.26589
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[150]	train's rmse: 1.62356	valid's rmse: 3.26411
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[151]	train's rmse: 1.62031	valid's rmse: 3.26838
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[152]	train's rmse: 1.61297	valid's rmse: 3.26445
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[153]	train's rmse: 1.61062	valid's rmse: 3.2623
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[154]	train's rmse: 1.60578	valid's rmse: 3.26756
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[155]	train's rmse: 1.59634	valid's rmse: 3.2627
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1

In [8]:
best_params = study.best_params
best_params["random_state"] = RANDOM_STATE
best_params

{'lambda_l1': 0.0012609340076084059,
 'lambda_l2': 0.43279665064103284,
 'num_leaves': 225,
 'feature_fraction': 0.626837358917216,
 'bagging_fraction': 0.7348290110818364,
 'bagging_freq': 1,
 'min_child_samples': 5,
 'random_state': 10}

In [9]:
# LightGBMを利用するのに必要なフォーマットに変換
lgb_train = lgb.Dataset(x_train, y_train)

# チューニングしたハイパーパラメーターをフィット
optimised_model = lgb.train(best_params, lgb_train)

# LightGBM推論
y_pred = optimised_model.predict(x_test)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000181 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 809
[LightGBM] [Info] Number of data points in the train set: 323, number of used features: 13
[LightGBM] [Info] Start training from score 21.968731
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

In [10]:
# 評価
def calculate_scores(true, pred):
    """全ての評価指標を計算する

    Parameters
    ----------
    true (np.array)       : 実測値
    pred (np.array)       : 予測値

    Returns
    -------
    scores (pd.DataFrame) : 各評価指標を纏めた結果

    """
    scores = {}
    scores = pd.DataFrame(
        {
            "R2": r2_score(true, pred),
            "MAE": mean_absolute_error(true, pred),
            "MSE": mean_squared_error(true, pred),
            "RMSE": np.sqrt(mean_squared_error(true, pred)),
        },
        index=["scores"],
    )
    return scores

In [11]:
scores = calculate_scores(y_test, y_pred)
print(scores)

             R2       MAE        MSE     RMSE
scores  0.84626  2.714941  16.078259  4.00977
